In [ ]:
%load_ext autoreload
%autoreload 2

# Microsoft Buildings Footprints

## Create/Download

In [ ]:
import pandas as pd

location = 'Ukraine'
dataset_links = pd.read_csv(
    "https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv"  # noqa E501
)
links = dataset_links[dataset_links.Location == location].copy()

In [ ]:
# Convert string such as 50.9KB to bytes

def convert_to_bytes(size):
    if size[-2:] == "KB":
        return int(float(size[:-2]) * 1024)
    elif size[-2:] == "MB":
        return int(float(size[:-2]) * 1024 * 1024)
    elif size[-2:] == "GB":
        return int(float(size[:-2]) * 1024 * 1024 * 1024)
    elif size[-1] == 'B':
        return int(float(size[:-1]))
    else:
        return int(size)

links['bytes'] = links['Size'].apply(convert_to_bytes)
print(f'Total size of the dataset: {links["bytes"].sum() / 1024**3:.2f} GB')

In [ ]:
from src.data.buildings.microsoft import download_microsoft_footprint

download_microsoft_footprint(location='Ukraine')

## 2: Vectorize predictions

nb: could also rasterize buildings instead: e.g. https://code.usgs.gov/arc/rasterized-building-footprints

In [ ]:
import rioxarray as rxr
from src.constants import PROJECT_PATH, PREDS_PATH
import rasterio
from src.data.buildings.microsoft import quadkeys_in_shape, load_buildings_geo
from src.data.unosat import get_unosat_geometry

def read_fp_within_geo(fp, geo):

    with rasterio.open(fp) as src:
        wind = rasterio.windows.from_bounds(*geo.bounds, src.transform)
        # data = src.read(window=wind)
    xa = rxr.open_rasterio(fp).rio.isel_window(wind)
    return xa

In [ ]:
from shapely.geometry import box
geo = box(31.32567,51.50041, 31.38216,51.53648)# zoom top-right UKR6

# geo = get_unosat_geometry(aoi)
fp = PREDS_PATH / '240212/240212_global_ukraine_preds.tif'
assert fp.exists()
gdf = load_buildings_geo(geo)
xa = read_fp_within_geo(fp, geo).squeeze()
gdf.shape, xa.shape

### 2.1 With rasterstats zonal_stats

In [ ]:
from rasterstats import zonal_stats

df_stats = pd.DataFrame(zonal_stats(
    vectors=gdf.to_crs(xa.rio.crs),
    raster=xa.squeeze().values,
    affine=xa.rio.transform(),
    stats=['mean', 'std', 'min', 'max'],
    nodata=xa.rio.nodata
))
print(df_stats.shape)
df_stats.head()

### 2.2 With xagg

In [ ]:
import os
from src.constants import PROJECT_PATH

# need this before importing xagg
os.environ['ESMFMKFILE'] = str(PROJECT_PATH / 's1tsdd-env/lib/esmf.mk')

import xagg

ds = xa.to_dataset(name='preds')
weightmap = xagg.pixel_overlaps(ds, gdf, impl='dot_product', subset_bbox=False)
agg_out = xagg.aggregate(ds, weightmap)

In [ ]:
df_xagg = agg_out.agg
df_xagg.head()

### 2.3 Check differences between the two methods

In [ ]:
# merge gdf with df_xagg on building_id

gdf_xagg = gdf.merge(df_xagg, on='building_id')
gdf_xagg['preds'] = gdf_xagg.preds.apply(lambda x: float(x[0]))
gdf_xagg[['geometry', 'preds']].to_file('test_xagg.geojson', driver='GeoJSON')

In [ ]:
gdf_xagg[['geometry', 'preds']].head()

In [ ]:
df_stats.index.name='building_id'
df_stats.reset_index()
gdf_zonal = gdf.merge(df_stats.reset_index(), on='building_id')
gdf_zonal.to_file('test_zonal.geojson', driver='GeoJSON')

In [ ]:
ds['preds'].to_netcdf('test_xa.nc')

In [ ]:
gdf_zonal.head(10)

In [ ]:
gdf_xagg.head(10)

### 2.4 Custom method

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box
import warnings
import xarray as xr

from src.constants import PROCESSED_PATH

def vectorize_xarray(xa: xr.DataArray, gdf: gpd.GeoDataFrame):

    if len(xa.shape) != 2:
        xa = xa.squeeze()
        assert len(xa.shape) == 2, 'xarray should be 2D'

    # Construct dataframe with one geometry per pixel
    x,y,v = xa.x.values, xa.y.values, xa.values
    x,y = np.meshgrid(x,y)
    x,y,v = x.flatten(), y.flatten(), v.flatten()
    df = pd.DataFrame.from_dict({'x': x, 'y': y, 'v': v})
    gdf_pixels = gpd.GeoDataFrame(v, geometry=gpd.GeoSeries.from_xy(df.x, df.y), columns=['preds'], crs=xa.rio.crs)
    gdf_pixels.index.name = 'pixel_id'
    gdf_pixels.reset_index(inplace=True)

    # Buffer the pixels to get one polygon per pixel
    res = xa.rio.resolution()
    buffer = res[0] / 2 # half the pixel size, assuming square pixels
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=UserWarning)
        gdf_pixels['geometry'] = gdf_pixels.buffer(buffer, cap_style=3)
    print('Pixels vectorized.')

    # Intersect the pixels with the buildings
    overlap = gpd.overlay(gdf, gdf_pixels, how='intersection')
    print('Pixels intersected with polygons.')

    # Calculate the value of the band at each polygon as the weighted area of the intersected pixels
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore',category=UserWarning)
        overlap['polygon_area'] = overlap.area
    preds_agg = overlap.groupby('building_id').apply(
        lambda row: (row['preds'] * row['polygon_area']).sum() / row['polygon_area'].sum()
    ).reset_index(name='preds_agg')
    print('Prediction aggregated.')

    return preds_agg

In [ ]:
geo = box(31.32567,51.50041, 31.38216,51.53648)# zoom top-right UKR6
aoi = 'UKR3'
geo = get_unosat_geometry(aoi)
fp = PROCESSED_PATH / 'settlements_predictions' / '240212' / '240212_global_ukraine_preds.tif'

gdf = get_buildings_geo(geo)
preds = read_fp_within_geo(fp, geo).squeeze()

In [ ]:
preds_agg = vectorize_xarray(preds, gdf)
print(preds_agg.shape)
preds_agg.head()